# Model 1
Baseline model using random forest

#### Import dataset

In [1]:
import pandas as pd

In [52]:
train_dataset = pd.read_csv("../data/train.csv")
train_dataset['date'] = pd.to_datetime(train_dataset['date'])

In [56]:
train_dataset.head(10)

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10
5,2013-01-06,1,1,12
6,2013-01-07,1,1,10
7,2013-01-08,1,1,9
8,2013-01-09,1,1,12
9,2013-01-10,1,1,9


#### Preprocessing

In [54]:
def aggregateByWeek(df, groupby_cols=['store','item'], sales_col='sales', date_col='date'):

    df = df.copy()
    #Convert to datetime, else throw an error
    df[date_col] = pd.to_datetime(df[date_col])

    #Add week and year columns
    df["startofweek"] = df[date_col] - pd.to_timedelta(df[date_col].dt.weekday, unit="D")
    df["year"] = df["startofweek"].dt.isocalendar().year
    df["week"] = df["startofweek"].dt.isocalendar().week

    df = df.groupby(by= groupby_cols + ["startofweek"], as_index=False)\
           .agg(
                year=("year", "min"),
                week=("week", "min"),
                sales=(sales_col, "sum"),
            )
    
    return df


In [55]:
weekly_sales = aggregateByWeek(train_dataset)
weekly_sales.head(10)

,store,item,startofweek,year,week,sales
0,1,1,2012-12-31,2013,1,73
1,1,1,2013-01-07,2013,2,66
2,1,1,2013-01-14,2013,3,80
3,1,1,2013-01-21,2013,4,70
4,1,1,2013-01-28,2013,5,86
5,1,1,2013-02-04,2013,6,82
6,1,1,2013-02-11,2013,7,80
7,1,1,2013-02-18,2013,8,78
8,1,1,2013-02-25,2013,9,83
9,1,1,2013-03-04,2013,10,97


In [33]:
def addLagColumns(df, groupby_cols=['store','item'], 
                  sales_col='sales', 
                  num_weeks_lag=3,
                  drop_null_rows=False):

    assert 0 < num_weeks_lag <= 10, "num_weeks_lag should be betwee 1 and 10 inclusive"

    df = df.copy()

    for i in range(1, num_weeks_lag+1):
        df[f"sales_{i}_wk_ago"] = df.groupby(by=groupby_cols)[sales_col].shift(i)
    
    if drop_null_rows:
        df = df.dropna(subset=[f"sales_{i}_wk_ago"]).reset_index(drop=True)
        
    return df

In [57]:
weekly_sales_with_lag = addLagColumns(df=weekly_sales,
                                      num_weeks_lag=7
                                      )
weekly_sales_with_lag.head(10)

,store,item,startofweek,year,week,sales,sales_1_wk_ago,sales_2_wk_ago,sales_3_wk_ago,sales_4_wk_ago,sales_5_wk_ago,sales_6_wk_ago,sales_7_wk_ago
0,1,1,2012-12-31,2013,1,73,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,2013-01-07,2013,2,66,73.0,NaN,NaN,NaN,NaN,NaN,NaN
2,1,1,2013-01-14,2013,3,80,66.0,73.0,NaN,NaN,NaN,NaN,NaN
3,1,1,2013-01-21,2013,4,70,80.0,66.0,73.0,NaN,NaN,NaN,NaN
4,1,1,2013-01-28,2013,5,86,70.0,80.0,66.0,73.0,NaN,NaN,NaN
5,1,1,2013-02-04,2013,6,82,86.0,70.0,80.0,66.0,73.0,NaN,NaN
6,1,1,2013-02-11,2013,7,80,82.0,86.0,70.0,80.0,66.0,73.0,NaN
7,1,1,2013-02-18,2013,8,78,80.0,82.0,86.0,70.0,80.0,66.0,73.0
8,1,1,2013-02-25,2013,9,83,78.0,80.0,82.0,86.0,70.0,80.0,66.0
9,1,1,2013-03-04,2013,10,97,83.0,78.0,80.0,82.0,86.0,70.0,80.0


In [58]:
def add_features(df,
                 groupby_cols=['store','item'], 
                 sales_col='sales', 
                 mean=True,
                 median=True,
                 stdev=True,
                 window_size=4
                 ):
    
    assert window_size >= 2, "window_size must be at least 2"

    df = df.copy()

    if mean:
        df[f"running_mean_{window_size}_wk"] = \
            df.groupby(by=groupby_cols)[sales_col]\
                .transform(lambda x: x.shift(1).rolling(window_size).mean())
    
    if median:
        df[f"running_median_{window_size}_wk"] = \
            df.groupby(by=groupby_cols)[sales_col]\
                .transform(lambda x: x.shift(1).rolling(window_size).median())
    
    if stdev:
        df[f"running_std_{window_size}_wk"] = \
            df.groupby(by=groupby_cols)[sales_col]\
                .transform(lambda x: x.shift(1).rolling(window_size).std())
    
    return df

In [59]:
weekly_sales_with_rolling_stats = add_features(weekly_sales_with_lag)
weekly_sales_with_rolling_stats.head(10)

,store,item,startofweek,year,week,sales,sales_1_wk_ago,sales_2_wk_ago,sales_3_wk_ago,sales_4_wk_ago,sales_5_wk_ago,sales_6_wk_ago,sales_7_wk_ago,running_mean_4_wk,running_median_4_wk,running_std_4_wk
0,1,1,2012-12-31,2013,1,73,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,1,2013-01-07,2013,2,66,73.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,1,2013-01-14,2013,3,80,66.0,73.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,1,2013-01-21,2013,4,70,80.0,66.0,73.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,1,2013-01-28,2013,5,86,70.0,80.0,66.0,73.0,NaN,NaN,NaN,72.25,71.5,5.909033
5,1,1,2013-02-04,2013,6,82,86.0,70.0,80.0,66.0,73.0,NaN,NaN,75.50,75.0,9.146948
6,1,1,2013-02-11,2013,7,80,82.0,86.0,70.0,80.0,66.0,73.0,NaN,79.50,81.0,6.806859
7,1,1,2013-02-18,2013,8,78,80.0,82.0,86.0,70.0,80.0,66.0,73.0,79.50,81.0,6.806859
8,1,1,2013-02-25,2013,9,83,78.0,80.0,82.0,86.0,70.0,80.0,66.0,81.50,81.0,3.415650
9,1,1,2013-03-04,2013,10,97,83.0,78.0,80.0,82.0,86.0,70.0,80.0,80.75,81.0,2.217356


#### Training and evaluating model